In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

model = ConcreteModel()

model.x = Var([1,2], domain=NonNegativeReals)

model.OBJ = Objective(expr = 2*model.x[1] + 3*model.x[2])

model.Constraint1 = Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)
opt = SolverFactory("glpk")
res = opt.solve(model, tee=True)
print "Solution:"
model.x.display()

GLPSOL: GLPK LP/MIP Solver, v4.57
Parameter(s) specified in the command line:
 --write /tmp/tmpSfpc4u.glpk.raw --wglp /tmp/tmpGD8o9u.glpk.glp --cpxlp /tmp/tmpdqSKkN.pyomo.lp
Reading problem data from '/tmp/tmpdqSKkN.pyomo.lp'...
2 rows, 3 columns, 3 non-zeros
21 lines were read
Writing problem data to '/tmp/tmpGD8o9u.glpk.glp'...
15 lines were written
GLPK Simplex Optimizer, v4.57
2 rows, 3 columns, 3 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  3.000e+00  max|aij| =  4.000e+00  ratio =  1.333e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00 inf =   1.000e+00 (1)
      1: obj =   7.500000000e-01 inf =   0.000e+00 (0)
*     2: obj =   6.666666667e-01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40416 bytes)
Writing basic solution to '/tmp/tmpSfpc4u.glpk.raw'...
7 lines were written
Solution:
x : Size=2, Index=x_index, Dom

In [3]:
from pyomo.environ import *
# create the concrete model
model = ConcreteModel()
# set the data (native python data)
k1 = 5.0/6.0 # min-1
k2 = 5.0/3.0 # min-1
k3 = 1.0/6000.0 # m3/(gmol min)
caf = 10000.0 # gmol/m3
# create the variables
model.sv = Var(initialize = 1.0, within=PositiveReals)
model.ca = Var(initialize = 5000.0, within=PositiveReals)
model.cb = Var(initialize = 2000.0, within=PositiveReals)
model.cc = Var(initialize = 2000.0, within=PositiveReals)
model.cd = Var(initialize = 1000.0, within=PositiveReals)

# create the objective
model.obj = Objective(expr = model.cb, sense=maximize)
# create the constraints
model.ca_bal = Constraint(expr = (0 == model.sv * caf \
    - model.sv * model.ca - k1 * model.ca \
    - 2.0 * k3 * model.ca ** 2.0))
model.cb_bal = Constraint(expr=(0 == -model.sv * model.cb \
    + k1 * model.ca - k2 * model.cb))
model.cc_bal = Constraint(expr=(0 == -model.sv * model.cc \
    + k2 * model.cb))
model.cd_bal = Constraint(expr=(0 == -model.sv * model.cd \
    + k3 * model.ca ** 2.0))

In [6]:
opt = SolverFactory("ipopt",solver_io='nl')
opt.solve(model,tee=True)

ERROR:pyomo.opt:Solver (asl) returned non-zero return code (-11)
ERROR:pyomo.opt:See the solver log above for diagnostic information.


ERROR: "[base]/site-packages/pyomo/opt/base/solvers.py", 568, solve
	Solver (asl) returned non-zero return code (-11)
ERROR: "[base]/site-packages/pyomo/opt/base/solvers.py", 571, solve
	See the solver log above for diagnostic information.


ApplicationError: Solver (asl) did not exit normally

In [7]:
opt = SolverFactory("bonmin",solver_io='nl')
opt.solve(model,tee=True)

/opt/Bonmin-stable/bin/bonmin: error while loading shared libraries: libcoinmumps.so.1: cannot open shared object file: No such file or directory
ERROR:pyomo.opt:Solver (asl) returned non-zero return code (127)
ERROR:pyomo.opt:See the solver log above for diagnostic information.


ERROR: "[base]/site-packages/pyomo/opt/base/solvers.py", 568, solve
	Solver (asl) returned non-zero return code (127)
ERROR: "[base]/site-packages/pyomo/opt/base/solvers.py", 571, solve
	See the solver log above for diagnostic information.


ApplicationError: Solver (asl) did not exit normally

In [27]:
import sys
from pyomo.opt.blackbox import RealOptProblem

class RealProblem1(RealOptProblem):

    def __init__(self):
        RealOptProblem.__init__(self)
        self.lower=[0.0, -1.0, 1.0, None]
        self.upper=[None, 0.0, 2.0, -1.0]
        self.nvars=4

    def function_value(self, point):
        self.validate(point)
        return point.vars[0] - point.vars[1] + (point.vars[2]-1.5)**2 + (point.vars[3]+2)**4

problem = RealProblem1()


AttributeError: 'RealProblem1' object has no attribute 'component_data_objects'